In [1]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from scipy.ndimage import gaussian_filter1d
from scipy.stats import expon

import pandas as pd

matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
matplotlib.rcParams['font.size'] = 18

from scipy.optimize import curve_fit


In [2]:
def autocorrelation(series,from_mean=True):
    """Compute the normalized autocorrelation function of a time series."""
    if from_mean:
        series = np.array(series) - np.mean(series)  # Remove mean
    else:
        series = np.array(series)
    n = len(series)
    norm = np.var(series) * n  # Normalization factor
    result = np.correlate(series, series, mode="full") / norm  # Compute autocorrelation
    return result[n-1:] / result[n-1:][0]

In [5]:
def projection(x,y,vx,vy):
    r = np.sqrt(x**2 + y**2)
    v = np.sqrt(vx**2 + vy**2)

    # Avoid division by zero
    r[r == 0] = np.nan
    v[v == 0] = np.nan

    # Compute projection
    projection = (x * vx + y * vy) / (r * v)
    projection[np.isnan(projection)] = 0
    
    
    return projection

def crossings(projection,t):
    sign_changes = np.where(np.sign(projection[:-1]) != np.sign(projection[1:]))[0]

    # Extract times at which the sign changes
    t_crossings = t[sign_changes]

    # Compute time intervals between consecutive crossings
    time_intervals = np.diff(t_crossings)
    return time_intervals


def displacement(x,y,step):
    dx = x[step:] - x[:-step]  # Difference in x
    dy = y[step:] - y[:-step]  # Difference in y

    displacements = np.sqrt(dx**2 + dy**2)
    
    return displacements

In [3]:
def find_zero_crossing_lag(acf):
    """Find the first zero crossing lag of the autocorrelation function."""
    indices = np.where(np.sign(acf[:-1]) != np.sign(acf[1:]))[0]
    return indices[0] + 1 if len(indices) > 0 else None


In [4]:
def mov_not_mov(A,th,zero_th=False):
    """
    Calculate moving and non moving steps.
    
    Parameters
    -----------
    A = array with the values to be analyzed, it can be the smoothed speed or the speed 
    
    th = threshold (taken from the prob of moving and prob of not moving)
    
    Return
    -------
    
    tuple: 
    
    (ns,ms,signal) = (list with NOT moving steps, list with MOVING steps,binary signal (+1 if this step and the next is above threshold, 0 if below))
     
    """
    
    # mask is an array that in each element has True if the value of array A is less than the threshold
    if zero_th:
        mask = A > th
        signal = [] # binary signal
        n=0 # counter for non moving steps
        m=0 # counter for moving steps
        ns=[] # list with consecutive non moving steps
        ms = [] # list with consecutive moving steps
        
        for i in range(len(A)-1):
            
            if mask[i] == False and mask[i+1] == False: # if this value in A and the next are below the threshold
                
                # print('n=',n)
                if m!=0: # append the current running value of m if we changed from moving to non moving
                    
                    ms.append(m)
                m=0 # reset the counter for moving steps
                signal.append(0) # append 0 to the signal
                n+=1 # begin counting non moving steps
            elif mask[i] == True or mask[i+1] == True: 
                # same as before but with the opposite condition
                if n!=0:
                    
                    ns.append(n)
                n=0
                signal.append(1)
                m+=1
            if i+1 == len(A)-1: 
                # if we are in the last element of A, we append the last values of n,m and the signal
                if mask[i] == False or mask[i+1] == False:
                    signal.append(0)
                    ns.append(n)
                else:
                    signal.append(1)
                    ns.append(m)
    else:
        mask = A <= th
        
        signal = [] # binary signal
        n=0 # counter for non moving steps
        m=0 # counter for moving steps
        ns=[] # list with consecutive non moving steps
        ms = [] # list with consecutive moving steps
        
        for i in range(len(A)-1):
            
            if mask[i] == True and mask[i+1] == True: # if this value in A and the next are below the threshold
                
                # print('n=',n)
                if m!=0: # append the current running value of m if we changed from moving to non moving
                    
                    ms.append(m)
                m=0 # reset the counter for moving steps
                signal.append(0) # append 0 to the signal
                n+=1 # begin counting non moving steps
            elif mask[i] == False or mask[i+1] == False: 
                # same as before but with the opposite condition
                if n!=0:
                    
                    ns.append(n)
                n=0
                signal.append(1)
                m+=1
            if i+1 == len(A)-1: 
                # if we are in the last element of A, we append the last values of n,m and the signal
                if mask[i] == False or mask[i+1] == False:
                    signal.append(1)
                    ms.append(m)
                else:
                    signal.append(0)
                    ns.append(n)

    return ns,ms,signal,mask
            

# All trajectories

In [ ]:
videos = [2,3,4]
speeds_videos = []


for video in videos: 
    
    name = f'data/new-videos/video{video}/video{video}_wspeedsmooth.csv'
    df = pd.read_csv(name,index_col=0)
  
    ids = df['Track ID'].unique()
  
    for id in ids:
        traj = df[df['Track ID'] == id].sort_values('Frame')
        x,y = traj['x (micron)'].values,traj['y (micron)'].values
        vx,vy = traj['vx'].values,traj['vy'].values
        